<a href="https://colab.research.google.com/github/SanGyuk-Raccoon/DACON_1/blob/main/ver_6_find_other_variable.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import numpy as np
import pandas as pd   

In [48]:
# Load DATA
train = pd.read_csv("/content/drive/MyDrive/DACON/[월간 데이콘] 14. 신용카드 사용자 연체 예측 AI/train.csv")

In [4]:
train.head()

,index,gender,car,reality,child_num,income_total,income_type,edu_type,family_type,house_type,DAYS_BIRTH,DAYS_EMPLOYED,FLAG_MOBIL,work_phone,phone,email,occyp_type,family_size,begin_month,credit
0,0,F,N,N,0,202500.0,Commercial associate,Higher education,Married,Municipal apartment,-13899,-4709,1,0,0,0,NaN,2.0,-6.0,1.0
1,1,F,N,Y,1,247500.0,Commercial associate,Secondary / secondary special,Civil marriage,House / apartment,-11380,-1540,1,0,0,1,Laborers,3.0,-5.0,1.0
2,2,M,Y,Y,0,450000.0,Working,Higher education,Married,House / apartment,-19087,-4434,1,0,1,0,Managers,2.0,-22.0,2.0
3,3,F,N,Y,0,202500.0,Commercial associate,Secondary / secondary special,Married,House / apartment,-15088,-2092,1,0,1,0,Sales staff,2.0,-37.0,0.0
4,4,F,Y,Y,0,157500.0,State servant,Higher education,Married,House / apartment,-15037,-2105,1,0,0,0,Managers,2.0,-26.0,2.0


In [27]:
var_identi = ['gender', 'car', 'reality', 'income_total', 'DAYS_BIRTH', 'DAYS_EMPLOYED']
train.loc[train.duplicated(var_identi)]

,index,gender,car,reality,child_num,income_total,income_type,edu_type,family_type,house_type,DAYS_BIRTH,DAYS_EMPLOYED,FLAG_MOBIL,work_phone,phone,email,occyp_type,family_size,begin_month,credit
192,192,F,N,Y,0,135000.0,Working,Higher education,Married,House / apartment,-13299,-1628,1,0,0,0,Laborers,2.0,-14.0,2.0
210,210,F,Y,Y,1,225000.0,Working,Higher education,Separated,House / apartment,-14816,-1672,1,0,0,0,Core staff,2.0,0.0,1.0
319,319,F,N,N,1,225000.0,Commercial associate,Secondary / secondary special,Civil marriage,House / apartment,-12640,-399,1,0,0,0,Accountants,3.0,-21.0,2.0
328,328,M,Y,Y,1,270000.0,Working,Secondary / secondary special,Married,House / apartment,-14488,-1630,1,0,1,0,Laborers,3.0,-22.0,2.0
330,330,F,N,N,0,270000.0,State servant,Secondary / secondary special,Separated,House / apartment,-19363,-12332,1,0,1,1,Medicine staff,1.0,-18.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26447,26447,M,N,Y,2,99000.0,Working,Secondary / secondary special,Married,House / apartment,-14226,-1026,1,1,1,0,Laborers,4.0,-43.0,2.0
26448,26448,M,N,Y,0,292500.0,Commercial associate,Higher education,Married,House / apartment,-16280,-887,1,0,0,0,Laborers,2.0,-23.0,0.0
26449,26449,F,N,N,0,90000.0,Working,Secondary / secondary special,Married,House / apartment,-10498,-2418,1,1,1,0,NaN,2.0,-2.0,1.0
26451,26451,F,N,Y,0,202500.0,Working,Higher education,Married,House / apartment,-12831,-803,1,1,1,0,Accountants,2.0,-44.0,1.0


In [53]:
#################그룹 내 순위 구하기#################################################
data = train.sort_values('begin_month')

data['ord_avg'] = data.groupby( var_identi )['begin_month'].rank(method = 'average')
data['ord_min'] = data.groupby( var_identi )['begin_month'].rank(method = 'min')
data['ord_max'] = data.groupby( var_identi )['begin_month'].rank(method = 'max')
data['ord_first'] = data.groupby( var_identi )['begin_month'].rank(method = 'first')
data['ord_dense'] = data.groupby( var_identi )['begin_month'].rank(method = 'dense')

#data.sort_values(var_identi).head(20)
train = data.sort_values('index')
train.head()
######################################################################################

,index,gender,car,reality,child_num,income_total,income_type,edu_type,family_type,house_type,DAYS_BIRTH,DAYS_EMPLOYED,FLAG_MOBIL,work_phone,phone,email,occyp_type,family_size,begin_month,credit,ord_avg,ord_min,ord_max,ord_first,ord_dense
0,0,F,N,N,0,202500.0,Commercial associate,Higher education,Married,Municipal apartment,-13899,-4709,1,0,0,0,NaN,2.0,-6.0,1.0,5.0,5.0,5.0,5.0,5.0
1,1,F,N,Y,1,247500.0,Commercial associate,Secondary / secondary special,Civil marriage,House / apartment,-11380,-1540,1,0,0,1,Laborers,3.0,-5.0,1.0,6.5,6.0,7.0,7.0,5.0
2,2,M,Y,Y,0,450000.0,Working,Higher education,Married,House / apartment,-19087,-4434,1,0,1,0,Managers,2.0,-22.0,2.0,4.5,4.0,5.0,5.0,4.0
3,3,F,N,Y,0,202500.0,Commercial associate,Secondary / secondary special,Married,House / apartment,-15088,-2092,1,0,1,0,Sales staff,2.0,-37.0,0.0,2.0,2.0,2.0,2.0,2.0
4,4,F,Y,Y,0,157500.0,State servant,Higher education,Married,House / apartment,-15037,-2105,1,0,0,0,Managers,2.0,-26.0,2.0,1.0,1.0,1.0,1.0,1.0


In [97]:
def conti_by_cate(GROUP, CONTI, DATA, method) :
  group = GROUP
  var = CONTI
  vec = []
  if method == 'mean' :
    vec = DATA.groupby( group )[ var ].mean()
  elif method == 'min' :
    vec = DATA.groupby( group )[ var ].min()
  elif method == 'max' :
    vec = DATA.groupby( group )[ var ].max()
  elif method == 'skew' :
    vec = DATA.groupby( group )[ var ].skew()
  elif method == 'med' :
    vec = DATA.groupby( group )[ var ].median()
  idx = vec.index

  for i in range(len(vec)) :
    DATA.loc[DATA[group] == idx[i], f'{var}_by_{group}_{method}'] = vec[i]


In [98]:
train.head()

,index,gender,car,reality,child_num,income_total,income_type,edu_type,family_type,house_type,DAYS_BIRTH,DAYS_EMPLOYED,FLAG_MOBIL,work_phone,phone,email,occyp_type,family_size,begin_month,credit,ord_avg,ord_min,ord_max,ord_first,ord_dense,income_total_by_occyp_type
0,0,F,N,N,0,202500.0,Commercial associate,Higher education,Married,Municipal apartment,-13899,-4709,1,0,0,0,NaN,2.0,-6.0,1.0,5.0,5.0,5.0,5.0,5.0,NaN
1,1,F,N,Y,1,247500.0,Commercial associate,Secondary / secondary special,Civil marriage,House / apartment,-11380,-1540,1,0,0,1,Laborers,3.0,-5.0,1.0,6.5,6.0,7.0,7.0,5.0,180307.189827
2,2,M,Y,Y,0,450000.0,Working,Higher education,Married,House / apartment,-19087,-4434,1,0,1,0,Managers,2.0,-22.0,2.0,4.5,4.0,5.0,5.0,4.0,278735.249654
3,3,F,N,Y,0,202500.0,Commercial associate,Secondary / secondary special,Married,House / apartment,-15088,-2092,1,0,1,0,Sales staff,2.0,-37.0,0.0,2.0,2.0,2.0,2.0,2.0,174955.579756
4,4,F,Y,Y,0,157500.0,State servant,Higher education,Married,House / apartment,-15037,-2105,1,0,0,0,Managers,2.0,-26.0,2.0,1.0,1.0,1.0,1.0,1.0,278735.249654


In [104]:
###################### 개인의 신용카드 시작일 간 차이
# 처음은 0으로 설정
train = pd.read_csv("/content/drive/MyDrive/DACON/[월간 데이콘] 14. 신용카드 사용자 연체 예측 AI/train.csv")
data = train.sort_values('begin_month')
data['begin_month_diff'] = data.groupby( var_identi )['begin_month'].diff()
data['begin_month_diff'] = data['begin_month_diff'].fillna(0)
data.sort_values(var_identi).head(20)

,index,gender,car,reality,child_num,income_total,income_type,edu_type,family_type,house_type,DAYS_BIRTH,DAYS_EMPLOYED,FLAG_MOBIL,work_phone,phone,email,occyp_type,family_size,begin_month,credit,begin_month_diff
6693,6693,F,N,N,0,27000.0,Working,Higher education,Civil marriage,House / apartment,-14869,-5067,1,1,0,0,Managers,2.0,-31.0,2.0,0.0
1141,1141,F,N,N,0,31500.0,Pensioner,Secondary / secondary special,Married,House / apartment,-20836,365243,1,0,0,0,NaN,2.0,-12.0,2.0,0.0
23988,23988,F,N,N,0,33300.0,Pensioner,Secondary / secondary special,Married,Office apartment,-19605,365243,1,0,0,0,NaN,2.0,-41.0,2.0,0.0
6274,6274,F,N,N,0,33300.0,Pensioner,Secondary / secondary special,Married,Office apartment,-19605,365243,1,0,0,0,NaN,2.0,-41.0,2.0,0.0
2390,2390,F,N,N,0,33300.0,Pensioner,Secondary / secondary special,Married,Office apartment,-19605,365243,1,0,0,0,NaN,2.0,-41.0,2.0,0.0
23659,23659,F,N,N,0,33300.0,Pensioner,Secondary / secondary special,Married,Office apartment,-19605,365243,1,0,0,0,NaN,2.0,-26.0,2.0,15.0
16649,16649,F,N,N,0,33300.0,Pensioner,Secondary / secondary special,Married,Office apartment,-19605,365243,1,0,0,0,NaN,2.0,-22.0,2.0,4.0
9987,9987,F,N,N,0,33300.0,Pensioner,Secondary / secondary special,Married,Office apartment,-19605,365243,1,0,0,0,NaN,2.0,-19.0,2.0,3.0
2189,2189,F,N,N,0,33750.0,Pensioner,Higher education,Married,House / apartment,-19054,365243,1,0,0,0,NaN,2.0,-7.0,2.0,0.0
9657,9657,F,N,N,0,36000.0,Working,Secondary / secondary special,Separated,Municipal apartment,-14846,-154,1,1,0,0,Cleaning staff,1.0,-53.0,0.0,0.0


In [7]:
var_identi = ['gender', 'car', 'reality', 'income_total', 'DAYS_BIRTH', 'DAYS_EMPLOYED']
train.loc[train[var_identi].duplicated(keep = 'first')].sort_values('DAYS_BIRTH')

,index,gender,car,reality,child_num,income_total,income_type,edu_type,family_type,house_type,DAYS_BIRTH,DAYS_EMPLOYED,FLAG_MOBIL,work_phone,phone,email,occyp_type,family_size,begin_month,credit
18952,18952,F,N,Y,0,76500.0,Pensioner,Secondary / secondary special,Widow,House / apartment,-25140,365243,1,0,0,0,NaN,1.0,-11.0,0.0
24279,24279,F,N,Y,0,76500.0,Pensioner,Secondary / secondary special,Widow,House / apartment,-25140,365243,1,0,0,0,NaN,1.0,-49.0,0.0
26112,26112,F,N,Y,0,90000.0,Pensioner,Secondary / secondary special,Separated,House / apartment,-24970,365243,1,0,0,0,NaN,1.0,-34.0,1.0
21815,21815,F,N,Y,0,58500.0,Pensioner,Secondary / secondary special,Single / not married,House / apartment,-24946,365243,1,0,1,0,NaN,1.0,-18.0,1.0
10345,10345,F,N,N,0,126000.0,Pensioner,Higher education,Single / not married,House / apartment,-24914,365243,1,0,0,0,NaN,1.0,-46.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15199,15199,F,N,Y,0,135000.0,Commercial associate,Incomplete higher,Married,House / apartment,-8056,-433,1,0,0,0,Core staff,2.0,-11.0,2.0
20609,20609,F,N,Y,1,180000.0,Working,Secondary / secondary special,Married,With parents,-8041,-1419,1,0,1,1,Sales staff,3.0,-28.0,2.0
25458,25458,F,N,N,0,90000.0,Commercial associate,Secondary / secondary special,Single / not married,With parents,-7757,-900,1,0,0,0,Sales staff,1.0,-13.0,1.0
13980,13980,F,N,N,0,90000.0,Commercial associate,Secondary / secondary special,Single / not married,With parents,-7757,-900,1,0,0,0,Sales staff,1.0,-2.0,0.0


In [ ]:
train['cnt'] = 1
train.sort_values("begin_month").groupby( ['gender', 'car', 'reality', 'income_total', 'DAYS_BIRTH', 'DAYS_EMPLOYED']).sum()

index  ...  cnt
gender car reality income_total DAYS_BIRTH DAYS_EMPLOYED         ...     
F      N   N       27000.0      -14869     -5067           6693  ...    1
                   31500.0      -20836      365243         1141  ...    1
                   33300.0      -19605      365243        82947  ...    6
                   33750.0      -19054      365243         2189  ...    1
                   36000.0      -14846     -154            9657  ...    1
...                                                         ...  ...  ...
M      Y   Y       900000.0     -18337     -1847          46250  ...    3
                                -16923     -2537          57849  ...    5
                                -15344     -1101          66883  ...    3
                                -15295     -5498          50861  ...    3
                                -9889      -1000          64469  ...    4

[8725 rows x 10 columns]

In [ ]:
train.loc[train[['gender', 'car', 'reality', 'income_total', 'DAYS_BIRTH', 'DAYS_EMPLOYED']].duplicated(keep = False)].sort_values('DAYS_BIRTH').head(20)

,index,gender,car,reality,child_num,income_total,income_type,edu_type,family_type,house_type,DAYS_BIRTH,DAYS_EMPLOYED,FLAG_MOBIL,work_phone,phone,email,occyp_type,family_size,begin_month,credit,cnt
18952,18952,F,N,Y,0,76500.0,Pensioner,Secondary / secondary special,Widow,House / apartment,-25140,365243,1,0,0,0,NaN,1.0,-11.0,0.0,1
17245,17245,F,N,Y,0,76500.0,Pensioner,Secondary / secondary special,Widow,House / apartment,-25140,365243,1,0,0,0,NaN,1.0,-55.0,0.0,1
24279,24279,F,N,Y,0,76500.0,Pensioner,Secondary / secondary special,Widow,House / apartment,-25140,365243,1,0,0,0,NaN,1.0,-49.0,0.0,1
26112,26112,F,N,Y,0,90000.0,Pensioner,Secondary / secondary special,Separated,House / apartment,-24970,365243,1,0,0,0,NaN,1.0,-34.0,1.0,1
17632,17632,F,N,Y,0,90000.0,Pensioner,Secondary / secondary special,Separated,House / apartment,-24970,365243,1,0,0,0,NaN,1.0,-22.0,1.0,1
21815,21815,F,N,Y,0,58500.0,Pensioner,Secondary / secondary special,Single / not married,House / apartment,-24946,365243,1,0,1,0,NaN,1.0,-18.0,1.0,1
15666,15666,F,N,Y,0,58500.0,Pensioner,Secondary / secondary special,Single / not married,House / apartment,-24946,365243,1,0,1,0,NaN,1.0,-36.0,1.0,1
957,957,F,N,N,0,126000.0,Pensioner,Higher education,Single / not married,House / apartment,-24914,365243,1,0,0,0,NaN,1.0,-51.0,0.0,1
10345,10345,F,N,N,0,126000.0,Pensioner,Higher education,Single / not married,House / apartment,-24914,365243,1,0,0,0,NaN,1.0,-46.0,2.0,1
14913,14913,F,N,N,0,90000.0,Pensioner,Secondary / secondary special,Single / not married,House / apartment,-24843,365243,1,0,0,0,NaN,1.0,-29.0,1.0,1


In [ ]:
# train 중복이 있는 경우 : 23251
# train 중복이 없는 경우 : 3206
print(train.loc[train[['gender', 'car', 'reality', 'income_total', 'DAYS_BIRTH', 'DAYS_EMPLOYED']].duplicated(keep = False)].shape)
print(train.loc[ ~ train[['gender', 'car', 'reality', 'income_total', 'DAYS_BIRTH', 'DAYS_EMPLOYED']].duplicated(keep = False)].shape)
# test 중복이 있는 경우 : 6889
# test 중복이 없는 경우 : 3111
print(test.loc[test[['gender', 'car', 'reality', 'income_total', 'DAYS_BIRTH', 'DAYS_EMPLOYED']].duplicated(keep = False)].shape)
print(test.loc[ ~ test[['gender', 'car', 'reality', 'income_total', 'DAYS_BIRTH', 'DAYS_EMPLOYED']].duplicated(keep = False)].shape)
# 전체 중복이 있는 경우 : 33383
# 전체 중복이 없는 경우 : 3074
data = pd.concat( [train, test] )
print(data.loc[data[['gender', 'car', 'reality', 'family_type', 'income_total', 'DAYS_BIRTH', 'DAYS_EMPLOYED', 'occyp_type', 'income_type']].duplicated(keep = False)].shape)
print(data.loc[ ~ data[['gender', 'car', 'reality', 'family_type', 'income_total', 'DAYS_BIRTH', 'DAYS_EMPLOYED', 'occyp_type', 'income_type']].duplicated(keep = False)].shape)

## 3111 -> 3074 : 37 건은 train에 이미 정보가 있음